<a href="https://colab.research.google.com/github/ZizZu94/nlu-second-assignment/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[NLU] Second Assignment** 
*   **Zihadul Azam**
*   Id: 221747
*   zihadul.azam@studenti.unitn.it




Commands to set Colab environment

In [27]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [28]:
import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/nlu/second-assignment/nlu-second-assignment')

In [32]:
!cd '/content/drive/My Drive/Colab Notebooks/nlu/second-assignment/nlu-second-assignment'

In [30]:
!pwd

/content


### **Requirements**


*   SpaCy: run `pip install spacy`
*   Sk

In [11]:
import re
import spacy
from spacy.tokens import Doc
import conll

from nltk.metrics import *

# utils
import utils

Load nlp and set white space tokenizer

In [12]:
nlp = spacy.load('en')

class WhitespaceTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(" ")
        return Doc(self.vocab, words=words)

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

#### **Global vars**

In [14]:
data_folder_path = './data'
train_path = data_folder_path + '/train.txt'
test_path = data_folder_path + '/test.txt'

#### **Load input data**
Load conll 2003 data from files and pre-process

In [15]:
def load_conll_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = f.read()
    sentences = data.split('\n\n')
    OUTPUT_DATA = []
    entities = []
    for sent in sentences:
        tokens = sent.split('\n')
        sentence = []
        ent_sentence_spacy = []
        ents = []

        if tokens[0] != '-DOCSTART- -X- -X- O' and tokens[0] != '':
            for x in tokens:
                x_split = x.split()
                # if not short length
                if len(x) > 0 and len(x_split) >= 3:
                    word = x_split[0]
                    word = word.strip()

                    if len(word) > 0:
                        sentence.append(word)
                        try:
                            ent = x_split[-1]
                        except IndexError:
                            print('Index Error: ', x_split)
                        ents.append((word, ent))
                # else:
                    #print('Short length x: ', x, ' . Removed.')

            processed_sentence = ' '.join(sentence)  # .lower()
            OUTPUT_DATA.append((processed_sentence, {'entities': ents}))

    print('Done getting data !')
    print('There are %d sentences.' % (len(sentences)))
    return OUTPUT_DATA

Mapper

In [33]:
def convert_spacy_entity_to_conll(iob, type):
    mapper = {
        'PERSON': 'PER',
        'NORP': 'MISC',
        'FAC': 'MISC',
        'ORG': 'ORG',
        'GPE': 'LOC',
        'LOC': 'LOC',
        'PRODUCT': 'MISC',
        'EVENT': 'MISC',
        'WORK_OF_ART': 'MISC',
        'LAW': 'MISC',
        'LANGUAGE': 'MISC',
        'DATE': 'MISC',
        'TIME': 'MISC',
        'PERCENT': 'MISC',
        'MONEY': 'MISC',
        'QUANTITY': 'MISC',
        'ORDINAL': 'MISC',
        'CARDINAL':  'MISC'
    }

    if not type in mapper:
        return 'O'
    return iob + '-' + mapper[type]

Get ref and hyp

In [34]:
def get_refs_hyps(data):
    index = 0
    total_data = len(data)

    refs = []
    hyps = []

    for sent in data:
        entities = sent[1]['entities']
        hyp = []
        doc = nlp(sent[0])
        for token in doc:
            conll_entity = convert_spacy_entity_to_conll(
                token.ent_iob_, token.ent_type_)

            hyp.append((token.text, conll_entity))
        index += 1

        # add this sentence refs
        sent_refs = [(entity[0], entity[1]) for entity in entities]
        refs.append(sent_refs)
        # add this sent hyp
        hyps.append(hyp)

        utils.printProgressBar(index, total_data,
                               prefix='Progress calculating accuracy:', suffix='Complete', length=50)
    return refs, hyps

# Task 1

In [35]:
conll_test_data = load_conll_file(test_path)
print('------------ First 2 sentences ------------')
for k in conll_test_data[:2]:
    print('Sentence: ', k[0])
    print('Entities:')
    for e in k[1]['entities']:
        print(e[0], '|', e[1])

    print('************************************')

print('\n')
print('------------ Task 1.1: report token-level performance (per class and total) ------------')
refs, hyps = get_refs_hyps(conll_test_data)
accuracy = conll.evaluate(refs, hyps)
print(accuracy)

entity_refs = []
for sent in refs:
    for token in sent:
        entity_refs.append(token[1])

entity_hyps = []
for sent in hyps:
    for token in sent:
        entity_hyps.append(token[1])

confusion_matrix = ConfusionMatrix(entity_refs, entity_hyps)
print(confusion_matrix)

FileNotFoundError: ignored